In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from src import Spotify
from src.utils import KMeans, DB

# My Data Setting

In [2]:
my_sp = Spotify()
my_sp.search()

계속하시려면 아무거나, 종료는 exit를 입력해주세요.exit


In [3]:
db = DB()
save_mb = db.save_mailbox(my_sp.sel_tracks)

# Recommender Init
- 실제로는 DB에 들어가서 Recommender가 빼는 형태라서, my_sp를 Recommender에 넣어서 빼는 작업이 필요

In [60]:
import pandas as pd
import numpy as np

In [47]:
save_mb.inserted_id

ObjectId('6265f7aafe9fef932765c60b')

## init_setting
- Spotify 로 부터 추천 음악들을 긁어오는 과정

In [204]:
mail_box_id = str(save_mb.inserted_id)
mail_box = db.get_mailbox(mail_box_id)

cols = ['trackId', 'trackName', 'artistIds', 'artistNames', 'image']

_sel_tracks = mail_box['tracks']
sel_tracks = pd.DataFrame(_sel_tracks)[cols]

sp = Spotify(sel_tracks)
sp.get_genres()
sp.get_features()
sp.get_recommend()

reco_sp = Spotify(sp.reco_tracks)
reco_sp.get_features()

my_tracks = sp.sel_tracks
my_features = sp.features

reco_tracks = reco_sp.sel_tracks
reco_features = reco_sp.features

## Data Preprocessing

In [205]:
features = pd.concat([my_features, reco_features])

norm_features = features.values[:, 1:]
norm_features = (norm_features - norm_features.min(axis=0)) / \
                (norm_features.max(axis=0) - norm_features.min(axis=0))

norm_features

array([[0.24263674614305752, 0.716824644549763, 0.7272727272727273, ...,
        0.09195859872611464, 0.39045231855987245, 0.19603742746822164],
       [0.5890603085553998, 0.7464454976303317, 0.45454545454545453, ...,
        0.06488853503184713, 0.9612623903383845, 0.5735452605434265],
       [0.8036465638148669, 0.683649289099526, 0.09090909090909091, ...,
        0.25159235668789803, 0.6525008545060955, 0.40196701186916095],
       ...,
       [0.48807854137447404, 0.7713270142180095, 0.09090909090909091,
        ..., 0.2914012738853503, 0.10903497778284152,
        0.45857213209526737],
       [0.6507713884992987, 0.6895734597156398, 0.09090909090909091, ...,
        0.46125265392781317, 0.45881280619801756, 0.430637137697968],
       [0.5988779803646564, 0.0, 0.18181818181818182, ...,
        0.3723460721868365, 0.30614105047282675, 0.3015038076953273]],
      dtype=object)

# Reco KMeans
- music filtering의 역할이 관건
- 너무 많이 추천되거나, 너무 적게 추천되는 경우

In [207]:
kmeans = KMeans(norm_features)
kmeans.fit()

ECV 46%
ECV 52%
ECV 53%
ECV 53%
ECV 54%
ECV 54%
ECV 54%
ECV 54%
ECV 55%
ECV 55%
ECV 55%
ECV 55%
ECV 55%
ECV 55%
ECV 55%


In [208]:
labels_ = kmeans.labels_
clusters_ = kmeans.clusters_

cluster_res = pd.DataFrame(
    np.column_stack([features['trackId'], labels_]),
    columns=['trackId', 'label']
)

my_labels = np.unique(pd.merge(my_features,cluster_res, on='trackId')['label'].values)

In [209]:
label_chk = np.isin(cluster_res['label'], my_labels)
recos = cluster_res[label_chk].copy()

my_tracks_chk = np.isin(recos['trackId'], my_tracks)
recos = recos[~my_tracks_chk]
recos

,trackId,label
2,18K3144Ugca0LSMqEi9KTw,6
7,7bp5DfkdK1OAvNJ1U4HfDA,6
8,4AtDiqnMxDt8crflZWgZG5,8
10,0c7QMo952Ga2qAJMdfEcrb,6
12,0gHcxtyWQT0HrlGxaxP1KT,6
...,...,...
166,5dFXrbiBhX3bWwAV4q4T1x,6
167,1EvB6tE5CHluLZaWm3r0bT,6
169,5V3cwxCMYUHCf8dtWgbw7e,6
174,3H1WufSXLnOPckdJnFDbtF,8


### 많을 경우

In [210]:
MAX_COUNT = 30

if len(recos) > MAX_COUNT:
    while True:
        cnt_df = recos.groupby("label").count()

        max_label = cnt_df['trackId'].idxmax()
        chk_idxes = recos[recos['label'] == max_label].index

        chk_cluster = np.expand_dims(clusters_[max_label], axis=0)
        chk_features = norm_features[chk_idxes]

        max_idx = euc(chk_cluster, chk_features).argmax()
        recos.drop(chk_idxes[max_idx], inplace=True)

        if len(recos) <= MAX_COUNT:
            break
#     break
            
reco_tracks_chk = np.isin(reco_tracks['trackId'], recos['trackId'])
print("추천 노래 개수 : {}개".format(len(reco_tracks[reco_tracks_chk])))
reco_tracks[reco_tracks_chk].head()

추천 노래 개수 : 30개


,trackId,trackName,artistIds,artistName,image
0,18K3144Ugca0LSMqEi9KTw,Habibi,5DIi2JWfQPTKffaVBlIYRn,Sik-K,https://i.scdn.co/image/ab67616d0000b273487582...
6,4AtDiqnMxDt8crflZWgZG5,Turn Off Your Phone (Feat. ELO) - Remix,4XDi67ZENZcbfKnvMnTYsI,Jay Park,https://i.scdn.co/image/ab67616d0000b27382ecc5...
13,5kqIPrATaCc2LqxVWzQGbk,7 Years,25u4wHJWxCA9vO0CzxAbK7,Lukas Graham,https://i.scdn.co/image/ab67616d0000b2732d94d0...
15,0lpRp9KzzMbC0w1uiL7H7f,Fruit Punch,6y02TEMv71ArWB2qhIaQ5m,Kaiydo,https://i.scdn.co/image/ab67616d0000b2731927f0...
16,7BKLCZ1jbUBVqRi2FVlTVw,Closer,"69GGBxA162lTqCwzJG5jLp,26VFTg2z8YR0cCuwLzESi2","The Chainsmokers,Halsey",https://i.scdn.co/image/ab67616d0000b273495ce6...


### 적을 경우

In [217]:
MIN_COUNT = 50

if len(recos) < MIN_COUNT:
    reco_mem = np.array([])

    # Save trackIds
    reco_mem = np.append(reco_mem,recos['trackId'].values)
    reco_mem

    # 대상 제거
    reco_tracks = reco_tracks[~np.isin(reco_tracks['trackId'], reco_mem)]
    reco_features = reco_features[~np.isin(reco_features['trackId'], reco_mem)]

    MAX_COUNT = MIN_COUNT - len(reco_mem)